In [1]:
import string
import numpy as np
import pandas as pd

In [2]:
df = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)
display(df.head())
print(df.shape)

df2 = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines=True)
display(df2.head())
print(df2.shape)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


(26709, 3)


,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


(28619, 3)


In [3]:
sum([i in df2.headline for i in df.headline])

0

In [4]:
df_all = pd.concat([df, df2]).reset_index(drop=True)
df_all.shape

(55328, 3)

In [5]:
df_clean = df_all.drop(['article_link'], axis=1)
df_clean.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [6]:
df_clean['headline'] = df_clean['headline'].apply(lambda x: x.lower())

In [7]:
df_clean['len'] = df_clean['headline'].apply(lambda x: len(x.split(" ")))
df_clean.head()

,headline,is_sarcastic,len
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14
3,"boehner just wants wife to listen, not come up...",1,13
4,j.k. rowling wishes snape happy birthday in th...,0,11


In [8]:
for i in string.punctuation:
    df_clean['headline'] = df_clean['headline'].apply(lambda x: x.replace(i, ""))

In [9]:
df_clean

,headline,is_sarcastic,len
0,former versace store clerk sues over secret bl...,0,12
1,the roseanne revival catches up to our thorny ...,0,14
2,mom starting to fear sons web series closest t...,1,14
3,boehner just wants wife to listen not come up ...,1,13
4,jk rowling wishes snape happy birthday in the ...,0,11
...,...,...,...
55323,jews to celebrate rosh hashasha or something,1,7
55324,internal affairs investigator disappointed con...,1,13
55325,the most beautiful acceptance speech this week...,0,12
55326,mars probe destroyed by orbiting spielberggate...,1,8


In [10]:
df_clean.groupby('is_sarcastic').describe()

len                                                  
                count       mean       std  min  25%   50%   75%    max
is_sarcastic                                                           
0             29970.0   9.818352  2.891726  2.0  8.0  10.0  12.0   38.0
1             25358.0  10.112943  3.697053  2.0  7.0  10.0  12.0  151.0

In [11]:
df_clean[df_clean.len==151].iloc[0].headline

'hot wheels ranked number one toy for rolling down ramp knocking over dominoes that send marble down a funnel dropping onto teetertotter that yanks on string causing pulley system to raise wooden block propelling series of twine rollers that unwind spring launching tennis ball across room inching tire down slope until it hits power switch activating table fan that blows toy ship with nail attached to it across kiddie pool popping water balloon that fills cup weighing down lever that forces basketball down track nudging broomstick on axis to rotate allowing golf ball to roll into sideways coffee mug which tumbles down row of hardcover books until handle catches hook attached to lever that causes wooden mallet to slam down on serving spoon catapulting small ball into cup attached by ribbon to lazy susan which spins until it pushes d battery down incline plane tipping over salt shaker to season omelet'

In [12]:
df_clean[df_clean.len==2]

,headline,is_sarcastic,len
416,breaking uniform,0,2
1042,refugee blues,0,2
1223,classic boring,1,2
1315,sunday roundup,0,2
1560,hes baaaaack,0,2
...,...,...,...
54274,celebrating christmas,0,2
54644,sunday roundup,0,2
54803,chaps unnecessary,1,2
55085,bye bye,0,2


In [13]:
sen = pd.read_csv('vader_lexicon.txt', 
                   sep='\t',
                   usecols=[0, 1], 
                   header=None, 
                   names=['token', 'polarity'],
                   index_col='token'
                  )
sen.head()

,polarity
token,
$:,-1.5
%),-0.4
%-),-1.5
&-:,-0.4
&:,-0.7


In [14]:
tidy_format = (
    df_clean['headline']
    .str.split(expand=True)
    .stack()
    .reset_index(level=1)
    .rename(columns={'level_1': 'num', 0: 'word'})
)
tidy_format.head()

,num,word
0,0,former
0,1,versace
0,2,store
0,3,clerk
0,4,sues


In [15]:
df_clean['polarity'] = (
    tidy_format
    .merge(sen, how='left', left_on='word', right_index=True)
    .reset_index()
    .loc[:, ['index', 'polarity']]
    .groupby('index')
    .sum()
    .fillna(0)
)
df_clean.head()

,headline,is_sarcastic,len,polarity
0,former versace store clerk sues over secret bl...,0,12,0.0
1,the roseanne revival catches up to our thorny ...,0,14,-1.3
2,mom starting to fear sons web series closest t...,1,14,-2.2
3,boehner just wants wife to listen not come up ...,1,13,0.0
4,jk rowling wishes snape happy birthday in the ...,0,11,4.2


In [16]:
df_clean.groupby('is_sarcastic').describe()

len                                                    \
                count       mean       std  min  25%   50%   75%    max   
is_sarcastic                                                              
0             29970.0   9.818352  2.891726  2.0  8.0  10.0  12.0   38.0   
1             25358.0  10.112943  3.697053  2.0  7.0  10.0  12.0  151.0   

             polarity                                                 
                count      mean       std   min  25%  50%  75%   max  
is_sarcastic                                                          
0             29970.0 -0.088916  2.132131 -11.2 -1.2  0.0  1.1  11.5  
1             25358.0 -0.073290  2.031312 -15.7 -1.2  0.0  1.1  11.3